<a href="https://colab.research.google.com/github/uba/tathu/blob/master/notebooks/system-detection-goes16.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Instalação TATHU - Tracking and Analysis of Thunderstorms**

![https://github.com/uba/tathu](https://raw.githubusercontent.com/uba/tathu/master/docs/sphinx/img/logo-tathu.png)



O código abaixo instala o pacote **condacolab** e reinicia o ambiente automaticamente. 

Nota: não é necessário rodar esse bloco novamente.

In [ ]:
!pip install -q condacolab
import condacolab
condacolab.install()

 Instalação dos pacotes necessários:

In [ ]:
!conda config --add channels conda-forge
!conda config --set channel_priority strict
!mamba install -c conda-forge aenum cartopy geopandas netcdf4 opencv pathos psycopg2 pyproj rasterstats requests rtree s3fs scikit-image scipy tqdm
!pip install pyyaml==5.4.1

Instalação do TATHU a partir do repositório oficial:

In [ ]:
!git clone https://github.com/uba/tathu.git

In [ ]:
%cd tathu/
!python -m pip install --no-deps .

Verificando a instalação:

In [ ]:
import tathu
print('Version', tathu.__version__)

from tathu.logo import TATHU_BANNER
print(TATHU_BANNER)

# **Download Canal 13 Full-Disk - GOES-16**

In [ ]:
from datetime import datetime
from tathu.downloader.goes import AWS
from tathu.progress import TqdmProgress

# Download 26 October 2022, Channel 13, [00] hours UTC
start = end = datetime.strptime('20221026', '%Y%m%d')
hours = ['00']

# From AWS (full-disk)
AWS.download(AWS.buckets['GOES-16'], ['ABI-L2-CMIPF'],
    start, end, hours, ['13'], './goes16-aws',
    progress=TqdmProgress('Download GOES-16 data (AWS)', 'files'))

# **Visualização Canal 13 Full-Disk - GOES-16**

In [ ]:
import matplotlib.pyplot as plt
from netCDF4 import Dataset

path = '/content/tathu/goes16-aws/noaa-goes16/ABI-L2-CMIPF/2022/299/00/OR_ABI-L2-CMIPF-M6C13_G16_s20222990000206_e20222990009525_c20222990010007.nc'

# Open netCDF file and show full-disk data
nc = Dataset(path)
data = nc.variables['CMI'][:]
plt.imshow(data, cmap='Greys', vmin=180.0, vmax=320.0)
plt.colorbar()
nc.close()

# **Remapeamento para Grade Regular**

In [ ]:
from osgeo import gdal

from tathu.constants import LAT_LONG_WGS84_SOUTH_AMERICA_EXTENT, LAT_LON_WGS84
from tathu.satellite import goes16

# Geographic area of regular grid
extent = LAT_LONG_WGS84_SOUTH_AMERICA_EXTENT

 # Grid resolution (kilometers)
resolution = 2.0

print('Remapping')
grid = goes16.sat2grid(path, extent, resolution, LAT_LON_WGS84, 'HDF5', progress=gdal.TermProgress_nocb)

# Visualize regular grid result
from tathu.visualizer import MapView

m = MapView(extent)
m.plotImage(grid, cmap='Greys', vmin=180.0, vmax=320.0)
m.show()


# **Detecção dos Sistemas Convectivos**

Definição dos parâmetros para deteção, incluindo valor máximo de temperatura de brilho (235K) e área mínima (3000km2): 

In [ ]:
from tathu.tracking.utils import area2degrees

# Threshold value
threshold = 235 # Kelvin
    
# Define minimum area
minarea = 3000 # km^2

# Convert to degrees^2
minarea = area2degrees(minarea)

Detecção utilizando processo de limiarização a partir da classe `detector.LessThan`:

In [ ]:
from tathu.tracking import detectors

# Create detector
detector = detectors.LessThan(threshold, minarea)

# Searching for systems
print('Searching for systems...')
systems = detector.detect(grid)
print('# Number of detected systems:', len(systems))

Visualização dos sistemas convectivos detectados:

In [ ]:
m = MapView(extent)
m.plotImage(grid, cmap='Greys', vmin=180.0, vmax=320.0)
m.plotSystems(systems, facecolor='none', edgecolor='red', centroids=False)
m.show()

# **Extração de Atributos dos Sistemas Convectivos**

Definição dos atributos que serão extraídos, incluindo valor mínimo, média, desvio padrão e número de pixels.



In [ ]:
# Attributes that will be computed
attrs = ['min', 'mean', 'std', 'count']

Extração de atributos utilizando a classe `descriptors.StatisticalDescriptor`:

In [ ]:
from tathu.tracking import descriptors

# Create statistical descriptor
descriptor = descriptors.StatisticalDescriptor(rasterOut=True)

# Describe systems (stats)
descriptor.describe(grid, systems)

# Silence some warnings in order to improve visualization
import warnings
from shapely.errors import ShapelyDeprecationWarning
warnings.filterwarnings('ignore', category=ShapelyDeprecationWarning) 

# Visualize attributes
for s in systems:
  print(s.name, s.attrs)